# Tutorial 1

more detailed docs: https://marctoussaint.github.io/robotics-course/

In [1]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
print(sys.version)
import numpy as np
import libry as ry

3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]
**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6

## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
C = ry.Config()
C.view()

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration.

In [3]:
C.clear()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

Note that the view was updated automatically.

In [4]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.5,.5,1.0])
ball.setColor([1,1,0])

In [5]:
C.delFrame(frameName="ball")

One can also add convex meshes (just passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [6]:
hand = C.addFrame(name="hand", parent="pr2L")
hand.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
hand.setRelativePosition([0,0,-.1])
hand.setColor([1,1,0])

In [7]:
C.delFrame(frameName="hand")

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

In [8]:
f = C.frame("hand")
print("position:", f.getPosition())
print("orientation:", f.getQuaternion())

**ry-c++-log** kin.cpp:getFrame:362(-1) cannot find frame named 'hand'

** WARNING:kin.cpp:getFrame:362(-1) cannot find frame named 'hand'


AttributeError: 'NoneType' object has no attribute 'getPosition'

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [9]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)

joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'torso_lift_joint', 'head_pan_joint', 'laser_tilt_mount_joint', 'r_shoulder_pan_joint', 'l_shoulder_pan_joint', 'head_tilt_joint', 'r_shoulder_lift_joint', 'l_shoulder_lift_joint', 'r_upper_arm_roll_joint', 'l_upper_arm_roll_joint', 'r_elbow_flex_joint', 'l_elbow_flex_joint', 'r_forearm_roll_joint', 'l_forearm_roll_joint', 'r_wrist_flex_joint', 'l_wrist_flex_joint', 'r_wrist_roll_joint', 'l_wrist_roll_joint', 'r_gripper_l_finger_jointr_gripper_r_finger_jointr_gripper_l_finger_tip_jointr_gripper_r_finger_tip_joint', 'l_gripper_l_finger_jointl_gripper_r_finger_jointl_gripper_l_finger_tip_jointl_gripper_r_finger_tip_joint', 'r_gripper_joint', 'l_gripper_joint']
joint state:  [ 0.          0.          1.57079633  0.1         0.          0.
 -1.          1.          0.4         0.5         0.5        -1.
  1.         -2.         -2.         -1.5         1.5        -0.5
 -0.5        -0.5 

Let's move the configuration by adding to the joint configuration

In [10]:
i = 20
q[i] = q[i] + 0.1
C.setJointState(q)

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [11]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  7.07106781e-01]
 [ 1.11022302e-17 -5.00000000e-02  8.90675000e-01 ...  5.00000000e-01
  -5.00000000e-01  5.00000000e-01]
 ...
 [-2.00000000e+00  0.00000000e+00  3.85000000e-01 ...  0.00000000e+00
   0.00000000e+00  7.07106781e-01]
 [-5.00000000e-01  2.00000000e+00  4.00000000e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.00000000e-01  2.00000000e+00  4.00000000e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


Let's do a questionable thing: adding .1 to all numbers in the frame matrix!

In [12]:
X = X0 + .1
C.setFrameState(X)

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [13]:
C.setJointState( C.getJointState() )

Now all *joint* transformations are consistent: just hingeX transformations or alike. However, all the other relative transformations of links and shapes are still messed up from setting their frame pose. Let's bring the configuration back into the state before the harsh *setFrame*

In [14]:
C.setFrameState(X0)

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

The frame state is not affected by such a selection of active joints.

In [15]:
C.selectJointsByTag(["armL","base"])
q = C.getJointState()
print('joint state: ', q)
print('joint names: ', C.getJointNames() )

joint state:  [ 0.          0.          1.57079633  1.00001     0.50001     1.00001
 -2.00001     1.50001    -0.50001     0.60001   ]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']


## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable features is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [16]:
F = C.feature(ry.FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [17]:
print(F.description(C))

[y,J] = F.eval(C)
print('hand position:', y)
print('Jacobian:', J)
print('Jacobian shape:', J.shape)

F_Position/0-pr2L
hand position: [-0.2316488   0.50511666  0.94627935]
Jacobian: [[ 1.00000000e+00  0.00000000e+00 -6.41448278e-01 -5.49219455e-01
   1.77226784e-01  9.83504202e-02 -2.78582737e-01 -2.44697071e-01
  -1.30902239e-02 -5.85555531e-02]
 [ 0.00000000e+00  1.00000000e+00 -4.07327096e-01 -4.97930545e-02
   6.09887383e-04  3.33188218e-02  4.11704323e-01  1.87401245e-01
  -5.29629980e-02  8.60963063e-02]
 [ 0.00000000e+00  0.00000000e+00  6.04018062e-02  6.59694140e-02
  -1.34575634e-01 -3.71888817e-01 -1.40042454e-01  9.63255100e-02
   1.98968136e-01 -2.46631576e-02]]
Jacobian shape: (3, 10)


We can linearly transform features by setting 'scale' and 'target':

In [250]:
F.setScale([10.]) #note: needs to be an array!
F.setTarget([0., 0., 1.])
F.eval(C)

(array([ 0.05116662, -0.68351204, -0.53720654]),
 array([[10.        ,  0.        , -3.76801955, -0.39831368,  0.22461843,
          0.28875259,  4.10799428,  0.82701635, -0.2895469 ,  0.92605822],
        [ 0.        , 10.        ,  3.05671735,  5.18643633,  0.14439229,
         -2.26107521,  2.42412134,  1.67721889,  0.32696353,  0.53079085],
        [ 0.        ,  0.        ,  1.2630568 , -0.09479733, -1.91901553,
         -4.24894324, -1.88015311,  1.23616929,  2.04357236, -0.63591961]]))

Setting scale and target actually transforms the feature to become
$$
  \phi(x) \gets \texttt{scale} \cdot (\phi(x) - \texttt{target})
$$
The $\cdot$ is flexibly interpreted: scale can be an arbitrary matrix, or a vector of same size as $\phi$ (element-wise re-scaling), or a single scalar (as above). E.g., if we only care about the height of the robot hand, we can do this:

In [18]:
F.setScale([[0,0,10]])      #pick z-coordinate only, and scale by 10
F.setTarget([0., 0., 1.])   #note that the target needs to be in UNSCALED space!
F.eval(C)

(array([-0.53720654]),
 array([[ 0.        ,  0.        ,  0.60401806,  0.65969414, -1.34575634,
         -3.71888817, -1.40042454,  0.9632551 ,  1.98968136, -0.24663158]]))

A negative-distance (penetration) feature example

In [19]:
F2 = C.feature(ry.FS.distance, ["hand", "ball"]) #distance actually means neg-distance
print(F2.description(C))

**ry-c++-log** kin.cpp:getFrame:362(-1) cannot find frame named 'hand'

**ry-c++-log** kin.cpp:getFrameIDs:393(-2) frame name 'hand' doesn't exist
** WARNING:kin.cpp:getFrame:362(-1) cannot find frame named 'hand'

STACK9 /usr/bin/python3(_PyEval_EvalCodeWithName
STACK8 /usr/bin/python3(_PyEval_EvalFrameDefault
STACK7 /usr/bin/python3() [0x50c383]
STACK6 /usr/bin/python3(_PyObject_MakeTpCall
STACK5 /usr/bin/python3(PyCFunction_Call
STACK4 /home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials/../build/libry.cpython-38-x86_64-linux-gnu.so(+0xb5cc8) [0x7f9ef4d78cc8]
STACK3 /home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials/../build/libry.cpython-38-x86_64-linux-gnu.so(+0x8a7e4) [0x7f9ef4d4d7e4]
STACK2 symbols2feature(FeatureSymbol, rai::Array<rai::String> const&, rai::Configuration const&, rai::Array<double> const&, rai::Array<double> const&, int)
STACK1 rai::Configuration::getFrameIDs(rai::Array<rai::String> const&) const
STACK0 rai::LogToken::~LogToken()


** ERROR:kin.cpp:getFrameIDs:393(-2) frame name 'hand' doesn't exist


RuntimeError: kin.cpp:getFrameIDs:393(-2) frame name 'hand' doesn't exist

In [257]:
F2.eval(C)

(array([-0.54596921]),
 array([[ 0.54425884,  0.8357221 ,  0.16099004,  0.51014398,  0.00276931,
         -0.25361415,  0.48137565,  0.27345866,  0.02537507,  0.15834579]]))

When a feature is of higher *order*, by default it computes the difference, acceleration, etc, w.r.t. multiple configurations. We need to create a compound configurations ("tuple") of several configurations to evaluate this:

In [20]:
C2 = ry.Config()
C2.addConfigurationCopy(C,tau=1.)  #this replicates the whole structure, but with semantics 'tau'-translated in time
C2.addConfigurationCopy(C,tau=1.)  #this replicates the whole structure
V2 = C2.view()

In [21]:
F.setScale([])  #reset to default (no scaling)
F.setTarget([]) #reset to default (zero target)
F.setOrder(1)
F.eval(C2)[0]

array([0., 0., 0.])

This should be zero, as the difference between the first and second copy is zero. To see a difference, let's move the 2nd configuration:

In [22]:
q = C.getJointState()
q = q - .1
C2.setJointStateSlice(q,1)
y = F.eval(C2)[0]
print('hand difference (y(C2) - y(K)) =', y)

hand difference (y(C2) - y(K)) = [-0.01637944  0.14782767 -0.04771976]


An acceleration example:

In [23]:
C2.addConfigurationCopy(C);

In [262]:
C2.setJointStateSlice(q + .3, 2);

In [24]:
F.setOrder(2)
(y,J) = F.eval(C2)
print('hand acceleration:', y)
print('shape of Jacobian:', J.shape)
print(F.description(C2))

hand acceleration: [ 0.01637944 -0.14782767  0.04771976]
shape of Jacobian: (3, 30)
F_Position/2-pr2L


Note that the Jacobian is now w.r.t. all three configurations! It is of size 3x(3xdim(q)). Let's retrieve the Jacobian w.r.t. C3 only:

In [25]:
J = J.reshape((3,3,q.size))
print('shape of Jacobian:', J.shape)
J[:,1,:]

shape of Jacobian: (3, 3, 10)


array([[ 1.00000000e+00,  0.00000000e+00, -4.57288990e-01,
        -4.88270516e-01, -7.41208944e-02,  2.29317675e-01,
        -1.77293745e-01, -3.23985662e-02, -4.61693975e-02,
        -3.46944695e-18],
       [ 0.00000000e+00,  1.00000000e+00, -1.15269355e-01,
         7.67830990e-02,  7.19858545e-02,  1.04320276e-02,
         4.66746632e-01,  1.15529727e-01,  2.43162706e-02,
         1.45716772e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -8.13133753e-17, -1.85096483e-01, -3.71887838e-01,
        -9.21214356e-03, -2.05797309e-02,  2.09190023e-01,
        -2.42861287e-16]])

In [26]:
C=0

## Editing Configurations

In [27]:
C = ry.Config()
C.view()

In [28]:
import subprocess
subprocess.Popen(['gedit', 'tmp.g'])

In [ ]:
## uncomment this! (Automatic testing for this interactive method does not work...)
C.edit('tmp.g')

**ry-c++-log** util.cpp:getIs:1025(-1) could not open file 'tmp.g' for input from '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials./.'
reloading `tmp.g' ... 


In [ ]:
C=0